In [130]:
import numpy as np
import random

## Trainingsdaten und DNF

In [131]:
# Trainingsdaten und DNF (Disjunktive Normalform)
# Die Trainingsdaten bestehen aus einer Matrix (dnf_training) und einer Liste von Eingaben mit zugehörigen Ausgaben (inputs_with_p)
dnf_training = np.array([
    [-1, -1, -1,  1, -1,  1, -1,  1,  0, -1],
    [-1,  0, -1,  1,  0, -1,  1,  1, -1,  0],
    [ 1, -1,  1,  0, -1, -1, -1,  1,  0,  0],
    [ 0, -1,  1,  0, -1, -1, -1,  1, -1, -1],
    [-1, -1,  0,  1, -1,  1,  1,  0,  0, -1]
]).astype(np.float64)

inputs_with_p = [
    (np.array([-1, -1, -1,  1, -1,  1, -1,  1,  1, -1]), 1),
    (np.array([-1,  1, -1,  1,  1, -1,  1,  1, -1,  1]), 1),
    (np.array([ 1, -1,  1,  1, -1, -1, -1,  1,  1,  1]), 1),
    (np.array([ 1, -1,  1,  1, -1, -1, -1,  1, -1, -1]), 1),
    (np.array([-1, -1,  1,  1, -1,  1,  1,  1,  1, -1]), 1),
    (np.array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]), -1),
    (np.array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1]), -1),
    (np.array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]), -1),
    (np.array([ 1, -1,  1, -1,  1, -1,  1, -1,  1, -1]), -1),
    (np.array([-1,  1, -1,  1, -1,  1, -1,  1, -1,  1]), -1)
]

## Aktivierungsfunktion (Signum-Funktion)

In [132]:
# Signum-Funktion (sig), die die Werte eines Vektors auf -1 oder 1 setzt
def sig(x):
    """
    Wendet die Signum-Funktion auf jeden Wert eines Vektors an.

    Parameter:
    - x: Eingabevektor (numpy-Array)

    Rückgabe:
    - Ein numpy-Array mit Werten -1 oder 1.
    """
    ret = np.copy(x)  # Kopiere den Eingabevektor
    for i in range(np.shape(x)[0]):  # Iteriere über alle Elemente
        if x[i] >= 0:
            ret[i] = 1  # Setze positive Werte auf 1
        else:
            ret[i] = -1  # Setze negative Werte auf -1
    return ret.astype(np.int64)  # Rückgabe als Integer-Array

## Hilfsfunktion

In [133]:
# Funktion zur Berechnung der Anzahl der verwendeten Literale in einem Monom
def get_used_literals(monome):
    """
    Zählt die Anzahl der nicht-null Werte in einem Monom.

    Parameter:
    - monome: Ein Array, das ein Monom repräsentiert

    Rückgabe:
    - Anzahl der verwendeten Literale (Werte ungleich 0).
    """
    literal_count = 0
    for elem in monome:
        if elem != 0:
            literal_count += 1  # Erhöhe den Zähler für jedes nicht-null Element
    return literal_count

## Schwellenwert zur Berechnung des Z-Layers

In [134]:
# Berechnung der Schwellenwerte für das Z-Layer
def calculate_treshold_z(w):
    """
    Berechnet die Schwellenwerte (v) für das Z-Layer basierend auf der Anzahl der verwendeten Literale.

    Parameter:
    - w: Gewichtsmatrix (numpy-Array)

    Rückgabe:
    - Ein numpy-Array mit Schwellenwerten.
    """
    v = np.empty(w.shape[0], dtype=np.float64)  # Initialisiere den Schwellenwert-Vektor
    for i in range(w.shape[0]):
        v[i] = get_used_literals(w[i]) - 0.5  # Berechnung des Schwellenwerts
    return v

## Berechnung z-Layer

In [135]:
# Berechnung des Z-Layers
def calculate_z(w, input, v):
    """
    Berechnet die Werte des Z-Layers.

    Parameter:
    - w: Gewichtsmatrix (numpy-Array)
    - input: Eingabevektor (numpy-Array)
    - v: Schwellenwerte (numpy-Array)

    Rückgabe:
    - Ein numpy-Array mit den berechneten Z-Werten.
    """
    z = np.empty(w.shape[0], dtype=np.float64)  # Initialisiere Z
    for j in range(w.shape[0]):
        z[j] = np.dot(w[j], input) - v[j]  # Skalarprodukt minus Schwellenwert
    z = sig(z)  # Wende die Signum-Funktion auf Z an
    return z

## Berechnung der Schwellenwerte und Gewichte für das y-Layer

In [136]:
# Berechnung der Schwellenwerte und Gewichte für das Y-Layer
def calculate_treshold_and_weights_y(z):
    """
    Berechnet die Schwellenwerte und Gewichte für das Y-Layer.

    Parameter:
    - z: Anzahl der Z-Werte (int)

    Rückgabe:
    - W: Gewichtsmatrix (numpy-Array)
    - V: Schwellenwerte (numpy-Array)
    """
    W = np.ones((z), np.float64)  # Initialisiere die Gewichtsmatrix mit Einsen
    V = -(z - 1)  # Berechnung der Schwellenwerte
    return W, V

## Berechnung des y-Layer

In [137]:
# Berechnung des Y-Layers
def calculate_y(W, V, z):
    """
    Berechnet die Ausgabe des Y-Layers.

    Parameter:
    - W: Gewichtsmatrix (numpy-Array)
    - V: Schwellenwerte (numpy-Array)
    - z: Eingabevektor für das Y-Layer (numpy-Array)

    Rückgabe:
    - y: Ausgabe des Y-Layers (int)
    """
    y = np.matmul(W, z.T)  # Matrixmultiplikation
    y = y - V  # Subtrahiere die Schwellenwerte
    if y >= 0:
        y = 1  # Setze positive Werte auf 1
    else:
        y = -1  # Setze negative Werte auf -1
    return y

## Test

In [138]:
input = inputs_with_p[0][0]  # Beispiel-Eingabevektor
print("Input:", input)

# Berechnung der Gewichtsmatrix W und der Schwellenwerte v
z_treshhold = calculate_treshold_z(dnf_training)
print("Tresholds:", z_treshhold)
#Berechne z
z = calculate_z(dnf_training, input, z_treshhold)
print("Z:", z)

#Berechnung der Gewichtsmatrix W und des Schwellenwerts V für die Ausgabe
y_treshhold = calculate_treshold_and_weights_y(z.shape[0])
print("W: ", y_treshhold[0])
print("V: ", y_treshhold[1])

# Berechne y
y = calculate_y(y_treshhold[0], y_treshhold[1], z)
print("y:", y)

Input: [-1 -1 -1  1 -1  1 -1  1  1 -1]
Tresholds: [8.5 6.5 6.5 7.5 6.5]
Z: [ 1 -1 -1 -1 -1]
W:  [1. 1. 1. 1. 1.]
V:  -4
y: 1


## Backpropagation

In [139]:
# Funktion zur Berechnung der Deltas für die Backpropagation
def calculate_deltas(input, p, lernrate, z, y, w, v, W, V):
    """
    Berechnet die Deltas für die Anpassung der Gewichte und Schwellenwerte während der Backpropagation.

    Parameter:
    - input: Eingabevektor (numpy-Array)
    - p: Erwartete Ausgabe (int)
    - lernrate: Lernrate (float)
    - z: Z-Werte (numpy-Array)
    - y: Ausgabe des Y-Layers (int)
    - w: Gewichtsmatrix für das Z-Layer (numpy-Array)
    - v: Schwellenwerte für das Z-Layer (numpy-Array)
    - W: Gewichtsmatrix für das Y-Layer (numpy-Array)
    - V: Schwellenwerte für das Y-Layer (numpy-Array)

    Rückgabe:
    - delta_w, delta_v, delta_W, delta_V: Anpassungen für die Gewichte und Schwellenwerte
    """
    delta_w = np.copy(w)  # Kopiere die Gewichtsmatrix
    delta_v = np.copy(v)  # Kopiere die Schwellenwerte
    delta_W = np.copy(W)  # Kopiere die Gewichtsmatrix für das Y-Layer
    delta_V = np.copy(V)  # Kopiere die Schwellenwerte für das Y-Layer

    error = p - y  # Berechne den Fehler

    for j in range(W.size):
        delta_W[j] = lernrate * error * z[j]  # Anpassung der Gewichte im Y-Layer
        delta_v[j] = -lernrate * error * W[j]  # Anpassung der Schwellenwerte im Z-Layer
        for k in range(input.size):
            delta_w[j][k] = lernrate * W[j] * error * input[k]  # Anpassung der Gewichte im Z-Layer
    delta_V = -lernrate * error  # Anpassung der Schwellenwerte im Y-Layer

    return delta_w, delta_v, delta_W, delta_V

In [140]:
""" # Test Backpropagation
p = inputs_with_p[0][1]  # Beispiel-Output
print(calculate_deltas(input, p, 0.1, z, y, dnf_training, z_treshhold, y_treshhold[0], y_treshhold[1])) """

' # Test Backpropagation\np = inputs_with_p[0][1]  # Beispiel-Output\nprint(calculate_deltas(input, p, 0.1, z, y, dnf_training, z_treshhold, y_treshhold[0], y_treshhold[1])) '

In [141]:
def change_weights(dnf, full):
    """
    Verändert die Gewichte in der DNF-Matrix.

    Parameter:
    - dnf: Gewichtsmatrix (numpy-Array)
    - full: Boolescher Wert, ob die Gewichte vollständig zufällig gesetzt werden sollen

    Rückgabe:
    - dnf: Angepasste Gewichtsmatrix
    """
    for i in range(dnf.shape[0]):
        for j in range(dnf.shape[1]):
            if not full:
                dnf[i][j] += (random.randint(-10, 10) * 0.1)  # Leichte Veränderung
            else:
                dnf[i][j] = random.randint(-10, 10) * 0.1  # Vollständige Zufälligkeit
    return dnf

In [142]:
#print(change_weights(dnf_training.astype(np.float64), True))

In [143]:
def feedforward_backpropagation(dnf, trainings_data, lerningsrate, epochs, mode):
    """
    Führt den Feedforward- und Backpropagation-Algorithmus aus.

    Parameter:
    - dnf: Gewichtsmatrix (numpy-Array)
    - trainings_data: Trainingsdaten (Liste von Tupeln)
    - lerningsrate: Lernrate (float)
    - epochs: Anzahl der Epochen (int)
    - mode: Modus zur Veränderung der Gewichte (0 = keine Veränderung, 1 = leichte Veränderung, 2 = zufällige Werte)
    """
    match mode:
        case 0:
            w = dnf  # Keine Veränderung der Gewichte
        case 1:
            w = change_weights(dnf, False)  # Leichte Veränderung der Gewichte
        case 2:
            w = change_weights(dnf, True)  # Zufällige Gewichte

    v = calculate_treshold_z(w)  # Berechnung der Schwellenwerte für das Z-Layer
    y_treshhold = calculate_treshold_and_weights_y(w.shape[0])  # Berechnung der Schwellenwerte und Gewichte für das Y-Layer
    W, V = y_treshhold

    for epoch in range(epochs):
        random_int = random.randint(0, len(trainings_data) - 1)  # Zufällige Auswahl eines Trainingsdatensatzes
        input = trainings_data[random_int][0]  # Eingabevektor
        p = trainings_data[random_int][1]  # Erwartete Ausgabe
        z = calculate_z(w, input, v)  # Berechnung der Z-Werte
        y = calculate_y(W, V, z)  # Berechnung der Ausgabe

        # Ausgabe der aktuellen Epoche und des Fehlers
        print(f"Epoche: {epoch + 1:03d} | Fehler: {p - y:3d} | y: {y:2d} | p: {p:2d}")

        # Berechnung der Deltas und Anpassung der Gewichte und Schwellenwerte
        delta_w, delta_v, delta_W, delta_V = calculate_deltas(input, p, lerningsrate, z, y, w, v, W, V)
        w = np.add(w, delta_w)
        v = np.add(v, delta_v)
        W = np.add(W, delta_W)
        V = np.add(V, delta_V)

In [144]:
# feedforward_backpropagation Test

#Modus 0 (nicht verändert)
#feedforward_backpropagation(dnf_training, inputs_with_p, 0.1, 200, 0)

#Modus 1 (leicht verändert)
feedforward_backpropagation(dnf_training, inputs_with_p, 0.1, 200, 1)

#Modus 2 (zufällige Werte)
#feedforward_backpropagation(dnf_training, inputs_with_p, 0.1, 200, 2)

Epoche: 001 | Fehler:   0 | y: -1 | p: -1
Epoche: 002 | Fehler:   0 | y: -1 | p: -1
Epoche: 003 | Fehler:   0 | y: -1 | p: -1
Epoche: 004 | Fehler:   0 | y: -1 | p: -1
Epoche: 005 | Fehler:   2 | y: -1 | p:  1
Epoche: 006 | Fehler:   0 | y:  1 | p:  1
Epoche: 007 | Fehler:   0 | y:  1 | p:  1
Epoche: 008 | Fehler:  -2 | y:  1 | p: -1
Epoche: 009 | Fehler:   0 | y: -1 | p: -1
Epoche: 010 | Fehler:   0 | y: -1 | p: -1
Epoche: 011 | Fehler:   0 | y: -1 | p: -1
Epoche: 012 | Fehler:   2 | y: -1 | p:  1
Epoche: 013 | Fehler:  -2 | y:  1 | p: -1
Epoche: 014 | Fehler:   2 | y: -1 | p:  1
Epoche: 015 | Fehler:  -2 | y:  1 | p: -1
Epoche: 016 | Fehler:   2 | y: -1 | p:  1
Epoche: 017 | Fehler:  -2 | y:  1 | p: -1
Epoche: 018 | Fehler:   2 | y: -1 | p:  1
Epoche: 019 | Fehler:   0 | y:  1 | p:  1
Epoche: 020 | Fehler:  -2 | y:  1 | p: -1
Epoche: 021 | Fehler:   0 | y: -1 | p: -1
Epoche: 022 | Fehler:   0 | y:  1 | p:  1
Epoche: 023 | Fehler:   0 | y: -1 | p: -1
Epoche: 024 | Fehler:   0 | y: -1 

## Auswertung
### Aufgabe a)
- um das Netz mit Backpropagation zu testen, wird "feedforward_backpropagation()" genutzt.
    - dazu muss eine DNF übergeben werden, mögliche Eingaben für die Variablen x, eine Lernrate, die Anzahl der Epochen und zuletzt der Modus, um Gewichte zu verändern.
### Aufgabe c) 
Wenn die Gewichte und Schwellenwerte initial so gesetzt werden, wie in der Aufgabenstellung beschrieben, ist die realisierung der DNF mit den gegebenen Variablen schon bei dem ersten Durchlauf des Netzes richtig. Das Ergebnis ist immer korrekt, was bedeutet, dass sich die Gewichte und Schwellwerte nicht mehr anpassen.

### Aufgabe d)
#### Mit leichter Veränderung der Schwellenwerte und Gewichte:

Mit leicht veränderten Werten macht das Netz von Anfang an wenig Fehler. Man erkennt, dass hier 

#### Mit zufälligen Schwellenwerten und Gewichten:

Das Nezt lernt tatsächlich mit der Zeit die Funktion richtig zu realisieren. Die meisten Fehler passieren in den ersten 25 Epochen vor, danach eher selten. Nach 50 - 100 Epochen kommen so gut wie keine Fehler vor. Allgemein kann man sagen, dass umso mehr Epochen das Netz durchgelaufen hat, umso weniger Fehler passieren.

